# Part I: 文法四元组提取 Grammatical quadruple extraction

In [1]:
grammar_string = "E ::= E + T | T\nT ::= T * F | F\nF ::= ( E ) | i"

### 定义文法提取函数

In [2]:
def add_production(non_terminal, production, productions):
    if non_terminal in productions:
        productions[non_terminal].append(production)
    else:
        productions[non_terminal] = [production]

def extract_from_line(s):
    left,right = s.split('::=')
    return left,right

def extract_rules_from_right(ls,rs,p):
    mid_re = rs.split('|')
    for item in mid_re:
        item_ts = []
        for item_t in item:
            item_ts += [item_t]
        add_production(ls,item_ts,p)

def extract_grammar_components(gs):
    production = {}
    start = ''
    terminators = []
    non_terminators = []
    gs = gs.replace(' ','')
    mid_s = gs.split("\n")
    for item_s in mid_s:
        left,right = extract_from_line(item_s)
        extract_rules_from_right(left, right, production)
        if ('Z' >= left >= 'A') and left not in terminators:
            terminators.append(left)
        if start == '':
            start = left
        for item_r in right:
            if item_r == '|': continue
            if (item_r > 'Z' or item_r < 'A') and item_r not in non_terminators:
                non_terminators.append(item_r)
    return terminators,non_terminators,production,start

**检验输出**

In [3]:
Vt,Vn,P,S = extract_grammar_components(grammar_string)
print(grammar_string,'\n')
print("Vt: ",Vt)
print("Vn: ",Vn)
print("P: ",P)
print("S: ",S)

# Part II 文法压缩 Grammatical compression

### 测试数据

In [4]:
extra_string = "Z ::= Be\nA ::= Ae\nA ::= e\nB ::= Ce\nB ::= Af\nC ::= Cf\nD ::= f\nE ::= E"

In [5]:
e_Vt,e_Vn,e_P,e_S = extract_grammar_components(extra_string)
print(extra_string,'\n')
print(e_Vt)
print(e_Vn)
print(e_P)
print(e_S)

### 删除U::=U 型的规则

In [6]:
def print_productions(productions):
    for key in productions:
        tss = ""
        for item in productions[key]:
            for item_s in item:
                for item_t in item_s:
                    tss += item_t
            tss += '|'
        print(f"{key}::=",tss[:-1])

In [7]:
def clean_UtoU(production):
    # 创建一个新的字典来保存修改后的产生式
    cleaned_production = {}
    for key, value in production.items():
        for item in value:
            if key in item and len(item) == 1:
                continue
            add_production(key, item, cleaned_production)
    return cleaned_production
# da = {'Z': [['B', 'e']], 'A': [['A', 'e'], ['e']], 'B': [['C', 'e'], ['A', 'f'],['B']], 'C': [['C', 'f']], 'D': [['D']]}
print_productions(clean_UtoU(e_P))

### 删除U::=u 型的规则
- 条件1：从S开始，对所有左部为有标记非终结符的生成式的右部的非终结符进行标记，反复进行这一过程至再无满足条件的生成式
- 条件2：对所有右部只有终结符的生成式的左部的非终结符进行标记。对所有左部为非终结符且右部只有终结符和标记非终结符的生成式的左部的非终结符进行标记，反复执行这一过程至再无满足条件的生成式
- 反复执行条件1和条件2的方法直至再无满足删除条件的生成式

In [8]:
import copy


def clean_Utou1(production, start):
    end_production = copy.deepcopy(production)  # 复制原始产生式字典
    marks = [start]
    start_list = production[start]
    # 遍历起始符号的产生式，标记所有出现在其中的非终结符
    for item in start_list:
        for item_s in item:
            if 'A' <= item_s <= 'Z' and item_s not in marks:
                marks.append(item_s)
    tag1 = True
    tag2 = True
    while tag1 or tag2:
        tag1 = False
        tag2 = False
        # 根据标记集合更新标记
        for key in production:
            if key in marks:
                continue
            k_list = production[key]
            for item in k_list:
                for item_s in item:
                    if 'A' <= item_s <= 'Z' and item_s in marks:
                        tag1 = True
                        marks.append(key)
        for key in marks:
            k_list = production[key]
            for item in k_list:
                for item_s in item:
                    if 'A' <= item_s <= 'Z' and item_s not in marks:
                        tag2 = True
                        marks.append(item_s)
    # 根据标记集合删除未标记的产生式
    for key in end_production.copy():
        if key not in marks:
            del end_production[key]
    return end_production

def clean_Utou2(production):
    end_production = {}
    marks = []
    for key in production:
        for item in production[key]:
            tag = True
            # 检测是否有非终结符
            for item_s in item:
                if 'A' <= item_s <= 'Z':
                    tag = False
                    break
            # 没有非终结符才能标记
            if tag: 
                marks.append(key)
                add_production(key,item,end_production)
    tag1 = True
    while tag1:
        tag1 = False
        for key in production:
            for item in production[key]:
                num_no_mark = False
                # 检测右部是否只有被标记的非终结符和终结符
                for item_s in item:
                    if 'A' <= item_s <= 'Z' and item_s not in marks:
                        num_no_mark = True
                        break
                # 右部只有被标记的非终结符和终结符时才能标记左部,当然前提是这条没被记录过
                if not num_no_mark and (key not in end_production or item not in end_production[key]):
                    tag1 = True
                    marks.append(key)
                    add_production(key,item,end_production)
    return end_production       

**检验输出**

In [9]:
print_productions(e_P)
print("")
clu = clean_Utou1(e_P,e_S)
print_productions(clu)

In [10]:
print_productions(clean_Utou2(clu))

### 自动化

In [11]:
def auto_clean(production,start):
    end_production = clean_UtoU(production)
    tag = True
    while tag:
        end_production = clean_Utou1(end_production,start)
        if end_production == end_production:
            tag = False
        end_production = clean_Utou2(end_production)
        if end_production == end_production:
            tag = False
    return end_production

检验输出

In [12]:
print_productions(e_P)
print('')
acl = auto_clean(e_P,e_S)
print_productions(acl)

# Part III 消除左递归 Eliminate left recursion

### 测试数据

In [13]:
left_recursion_string = "S ::= Sa | Tbc | Td\nT ::= Se | gh"
left_recursion_string_1 = "A ::= Ba | Cb | c\nB ::= da | Ae | f\nC ::= Bg | h"
print(left_recursion_string,'\n')
print(left_recursion_string_1)

In [14]:
l_Vt_1,l_Vn_1,l_P_1,l_S_1 = extract_grammar_components(left_recursion_string_1)
print(l_Vt_1)
print(l_Vn_1)
print(l_P_1)
print(l_S_1)

In [15]:
l_Vt,l_Vn,l_P,l_S = extract_grammar_components(left_recursion_string)
print(l_Vt)
print(l_Vn)
print(l_P)
print(l_S)

### 函数方法实现

In [16]:
def remove_production(non_terminal, production, productions):
    if non_terminal in productions and production in productions[non_terminal]:
        productions[non_terminal].remove(production)
        if not productions[non_terminal]:  # If the set becomes empty, remove the key
            del productions[non_terminal]

def eliminate_rules(key_item,value_item,production):
    end_production = copy.deepcopy(production)
    mid_key_item = key_item
    end_key_item = mid_key_item + "'"
    mid_value_item = value_item.copy()
    end_value_item = value_item.copy()
    # 先改后删
    for i in range(len(end_value_item)):
        if end_value_item[i] == mid_key_item:
            end_value_item.pop(i)
            end_value_item.append(end_key_item)
            break
    for item in end_production[mid_key_item]:
        if item == mid_value_item: continue
        if mid_key_item in item: continue
        if end_key_item in item: continue
        item.append(end_key_item)
    remove_production(mid_key_item,mid_value_item,end_production)
    add_production(end_key_item,end_value_item,end_production)
    if ['ε'] not in end_production[end_key_item]:
        add_production(end_key_item,['ε'],end_production)
    else:
        remove_production(end_key_item,['ε'],end_production)
        add_production(end_key_item,['ε'],end_production)
    return end_production
        
def auto_eliminate_rules(productions):
    end_productions = copy.deepcopy(productions)
    tag = True
    while tag:
        tag = False
        for key in end_productions:
            for item in end_productions[key]:
                # 消去规则左递归
                if key in item and len(key) == 1:
                    tag = True
                    end_productions = eliminate_rules(key,item,end_productions)  
    return end_productions


def eliminate_grammar(productions):
    mid_productions = copy.deepcopy(productions)
    end_productions = copy.deepcopy(productions)
    mid_changed_keys_list = []
    for key in productions:
        if "'" in key:
            mid_changed_keys_list.append(key.replace("'",''))
    flg = False
    for key in mid_productions:
        if key in mid_changed_keys_list: continue
        for item in mid_productions[key]:
            tag = False
            for i in range(len(item)):
                if item[i] in mid_changed_keys_list:
                    tag = True
                    flg = True
                    for item_i in mid_productions[item[i]]:
                        end_item = item.copy()
                        end_item.pop(i)
                        end_item[i:i] = item_i
                        add_production(key,end_item,end_productions)
            if tag:
                remove_production(key,item,end_productions)
    return end_productions,flg           

**检验输出**

In [17]:
# l_P = {'S': [['Q', 'c'], ['R', 'd'], ['c']],
#                   'Q': [['R', 'b'], ['S', 'e'], ['b']],
#                   'R': [['S', 'a'], ['Q', 'f'], ['a']]}
l_P = {'S': [['S', 'a'], ['T', 'b', 'c'], ['T', 'd']], 
       'T': [['S', 'e'], ['g', 'h']]}
print_productions(l_P)
print('\n')
l_Pr = auto_eliminate_rules(l_P)
print_productions(l_Pr)
print('\n')
l_Pg,flg = eliminate_grammar(l_Pr)
print_productions(l_Pg)
print(flg,'\n')
l_Pg2,flg = eliminate_grammar(l_Pg)
print_productions(l_Pg2)
print(flg,'\n')

### 自动化

In [18]:
def auto_eliminate_left_recursion(productions):
    end_productions = copy.deepcopy(productions)
    while True:  
        end_productions = auto_eliminate_rules(end_productions)
        end_productions,flg = eliminate_grammar(end_productions)
        if not flg:
            break
    return end_productions

**检验输出**

In [19]:
print_productions(l_P)
l_Pc = auto_eliminate_left_recursion(l_P)
print('')
print_productions(l_Pc)

In [20]:
print_productions(P)
Pc = auto_eliminate_left_recursion(P)
print('')
print_productions(Pc)

# Part IV 最左推导 Leftmost derivation

### 测试数据

In [21]:
print_productions(P)

In [22]:
derivation_string = "i + i * i"
derivation_string_1 = "( i + i ) * i"
print(derivation_string)
print(derivation_string_1)

### 构建推导方法函数

In [23]:
def input_judgment(derived_string, key, end_string,pos):
    while True:
        answer = input("\nDerive {} with {} -> {}? pos: {} (y/n): \n".format(derived_string, key, end_string,pos))
        if answer == 'y':
            return False
        elif answer == 'n':
            return True
        else:
            print("Please enter 'y' or 'n'")
            continue
    # 如果回答不是'y'，则跳过当前产生式


def find_value_in_list(lst, value):
    for i, item in enumerate(lst):
        if item == value:
            return i
    return -1


def auto_derivation(input_str, productions, start_symbol):
    derived_string = [start_symbol]  # 初始时将起始符号作为推导的起点
    result = ""  # 存储推导结果
    tree = {}  # 存储语法分析树
    # 逐步推导直到无法再推导为止
    while derived_string:
        derived = False  # 标志是否成功推导
        # 遍历所有产生式
        for key in productions:
            # 如果当前推导串中包含产生式左部，则问询是否进行替换
            if key in derived_string:
                for item in productions[key]:
                    # 问询是否替换
                    end_string = "".join(item)
                    mid_string = "".join(derived_string)

                    pos = find_value_in_list(derived_string, key)
                    if input_judgment(mid_string, key, end_string,pos + 1):
                        continue  # 如果回答不是'y'，则跳过当前产生式
                    
                    # 将当前推导过程添加到结果中
                    result += " => {} ({} -> {})\n".format(mid_string, key, end_string)

                    # 进行替换
                    derived_string[pos:pos+1] = item
                    derived = True  # 表示成功推导
                    
                    end_item = [pos + 1,item]
                    # 更新语法分析树
                    if key in tree:
                        tree[key].append(end_item)
                    else:
                        tree[key] = [end_item]
                    break  # 结束当前循环，重新开始检查推导串

                # 如果成功推导，就跳出外层循环
                if derived:
                    break

        # 如果无法再进行推导，则退出循环
        if derived_string == input_str:
            mid_string_out = "".join(derived_string)
            result += " => {}\n".format(mid_string_out)
            break
        if not derived:
            mid_string_out = "".join(derived_string)
            result += " => {}\n".format(mid_string_out)
            break

    return result, tree


检验输出

In [24]:
PP = {'Ems': [['Ems', '+', 'Tfs'], ['Tfs']], 'Tfs': [['Tfs', '*', "F'"], ["F'"]], "F'": [['(', 'Ems', ')'], ['iss']]}
SS = 'Ems'
DD = 'iss + iss + iss'
rel, tre = auto_derivation(DD, PP, SS)
print(rel)
print(tre)

# Part V 语法分析树构建 Parse Tree

### 函数方法实现

In [25]:
import copy

def insert_separator_uniformly(lst):
    n = len(lst)
    new_lst = [None] * (2 * n - 1)  # 创建一个新列表，长度为2n-1

    # 将原列表中的元素和23交替插入新列表中
    for i in range(n):
        new_lst[i * 2] = lst[i]
        if i < n - 1:
            new_lst[i * 2 + 1] = '_'*10

    return new_lst

def generate_tree(tree, start):
    mid_tree = copy.deepcopy(tree)
    pre_string = ["_" * 10] + [start]
    result = ''.join(pre_string) + '\n'
    tag = True
    while tag:
        tag = False
        # cur_string = []
        j = 0
        for i in range(len(pre_string)):
            if pre_string[i] == '_' * 10: 
                continue
            j += 1
            if pre_string[i] in mid_tree:
                tag = True
                item_left, item_rigth = mid_tree[pre_string[i]][0]
                if j == item_left:
                    mid_tree[pre_string[i]].pop(0)
                    pre_string[i:i+1] =  insert_separator_uniformly(item_rigth)
                    # cur_string = pre_string[:i] + insert_separator_uniformly(item_rigth) + pre_string[i + 1:]
                    break
        result += ''.join(pre_string) + '\n'
        # pre_string = cur_string
    return result


**检验输出**

In [26]:
# 测试
print("语法分析树：")
TT = {'Ems': [[1, ['Ems', '+', 'Tfs']], [1, ['Ems', '+', 'Tfs']], [1, ['Tfs']]], 'Tfs': [[1, ["F'"]], [3, ["F'"]], [5, ["F'"]]], "F'": [[1, ['iss']], [3, ['iss']], [5, ['iss']]]}
ans = generate_tree(TT, SS)
print(ans)

# Part VI 从正则文法构造有穷状态自动机 finite state automata

In [27]:
from extract_grammar import GrammaticalQuadrupleExtraction
# from data.grammar import grammar_str8
grammar_str8 = ("Z ::= Z a | A a | B b\n"
                "A ::= B a | Z a | a\n"
                "B ::= A b | B a | b")
Vt,Vn,P,S = GrammaticalQuadrupleExtraction().extract_grammar_components(grammar_str8)
print(grammar_str8,'\n')
print("Vt: ",Vt)
print("Vn: ",Vn)
print("P: ",P)
print("S: ",S)

In [28]:
def finite_state_automata(grammar_str):
    terminators, non_terminators,productions,start = GrammaticalQuadrupleExtraction().extract_grammar_components(grammar_str)
    states = terminators + ['My_end'] # 状态集合 K
    alphabet = non_terminators # 符号集合 Σ
    transition_function = {} # 转移函数 M
    start_state = ['My_end'] # 起始状态 S
    accept_state = start # 接受状态 F 
    
    matrix = {row_label: {col_label: None for col_label in alphabet} for row_label in states}
    for key in productions:
        for item in productions[key]:
            if len(item) == 1:
                matrix['My_end'][item[0]] = key
            if len(item) == 2:
                matrix[item[0]][item[1]] = key
    transition_function = matrix
    print(states)
    print(alphabet)
    print(transition_function)
    print(start_state)
    print(accept_state)

In [29]:
finite_state_automata(grammar_str8)

# 2.0
- 目前未考虑Z ::= aa 之类右部只有一个终结符的情况
- 目前未考虑右部长度大于二的情况

In [1]:
# 2.0
def finite_state_automata2(grammar_str):
    terminators, non_terminators,productions,start = GrammaticalQuadrupleExtraction().extract_grammar_components(grammar_str)
    states = terminators + ['My_end'] # 状态集合 K
    alphabet = non_terminators # 符号集合 Σ
    transition_function = {} # 转移函数 M
    start_state = ['My_end'] # 起始状态 S
    accept_state = [start] # 接受状态 F 
    
    matrix = {row_label: {col_label: None for col_label in alphabet} for row_label in states}
    for key in productions:
        for item in productions[key]:
            if len(item) == 1:# 右部只有一个终结符
                if ('My_end', item[0]) not in transition_function:
                    transition_function[('My_end', item[0])] = [key]
                else:
                    transition_function[('My_end', item[0])].append(key)
                # matrix['My_end'][item[0]] = key
            if len(item) == 2:
                if (item[0], item[1]) not in transition_function:
                    transition_function[(item[0], item[1])] = [key]
                else:
                    transition_function[(item[0], item[1])].append(key)
    return states, alphabet, transition_function, start_state, accept_state

In [2]:
def transition_function_print(transition_function):
    result = ''
    for row_label, row_data in transition_function.items():
        # print("M",row_label, "=", row_data)
        result += "M(" + ','.join(row_label) + ") = {" + ','.join(row_data) + '}\n'
    return result

In [3]:
from extract_grammar import GrammaticalQuadrupleExtraction
grammar_str8 = ("Z ::= Z a | A a | B b\n"
                "A ::= B a | Z a | a\n"
                "B ::= A b | B a | b")

In [4]:
# finite_state_automata2(grammar_str8)
states, alphabet, transition_function, start_state, accept_state = finite_state_automata2(grammar_str8)
print("K : {"+ ','.join(states)+'}')
print("Σ : {"+ ','.join(alphabet) + '}')
print("M : ")
print(transition_function)
print("S : {"+','.join(start_state)+'}')
print("F : {"+','.join(accept_state)+'}')

In [5]:
print(transition_function_print(transition_function))



## FA构建与确定化
- NFA->DFA
- DFA简化

### 尝试运行

In [6]:
class FiniteAutomaton:
    def __init__(self, K, sigma, M, S, F):
        self.states = set(K)  # 状态集合
        self.alphabet = set(sigma)  # 输入字母表
        self.transitions = M  # 状态转移函数
        self.start_state = S[0]  # 初始状态
        self.accept_states = F  # 接受状态集合

    def run(self, input_string):
        current_state = self.start_state
        for symbol in input_string:
            if (current_state, symbol) in self.transitions:
                mid_state = self.transitions[(current_state, symbol)]
                current_state = mid_state[0]
                # print(current_state)
            else:
                return False
        return current_state in self.accept_states


# 从给定的五元组构建有穷状态自动机
def construct_fsm(K, sigma, M, S, F):
    return FiniteAutomaton(K, sigma, M, S, F)



In [7]:
# 例子
# A ::= S 0 | A 1 | 0
# B ::= A 0 | B 0 | B 1 | 1
grammar_str81 = ("B ::= A 1 | B 0 | B 1 | 1\n"
                 "A ::= A 0 | 0\n")
K,sigma,M,S,F = finite_state_automata2(grammar_str81)
print(K,sigma,M,S,F)

# M = {('S', '0'): 'A', ('S', '1'): 'B', ('A', '0'): 'A', ('A', '1'): 'B', ('B', '0'): 'B', ('B', '1'):'B'}
# K = {'S', 'A', 'B'}
# S = 'S'
# F = {'B'}


# print(K,sigma,M,S,F)
# 给定的五元组
# sigma = {'0', '1'}



In [8]:
# M[('S', '0')][0]
# M = {('S', '0'): 'A', ('S', '1'): 'B', ('A', '0'): 'A', ('A', '1'): 'B', ('B', '0'): 'B', ('B', '1'): 'B'}
# M[('S', '0')]

In [9]:
# 构建有穷状态自动机
fsm = construct_fsm(K, sigma, M, S, F)


In [10]:
# 测试字符串
test_strings = ['010', '111', '1001', '00', '11']

# 运行有穷状态自动机并输出结果
for s in test_strings:
    print(f"String '{s}' matches: {fsm.run(s)}")

In [11]:
fsm = construct_fsm(states, alphabet, transition_function, start_state, accept_state)


In [12]:
transition_function[('Z', 'a')][0]

In [13]:
test_strings = ['ba','babb','babbabb']

for ts in test_strings:
    print(f"String '{ts}' matches: {fsm.run(ts)}")

## NFA勉强能跑，但是这是强制在有两个目标状态的情况下选择其一的无奈之举

### 接下来实现NFA到DFA的转换

In [14]:
print(transition_function)

In [15]:
import copy

tf = copy.deepcopy(transition_function)
new_tf = {}  # 创建一个新的字典来保存修改后的键值对

tag = True
while tag:
    # tag = False
    for key in tf:
        if len(tf[key]) != 1:
            # tag = True
            mid_key = "".join(tf[key])  # 将值拼接成一个字符串作为新键
            mid_key_list = tf[key]  # 将值列表作为新键的值
            new_values = {}
            for item in alphabet:
                list1 = []
                for sub_key in mid_key_list:
                    try:
                        list1 += tf[(sub_key, item)]
                    except KeyError:
                        continue
                unique_list = list(set(list1))
                new_tf[(mid_key,item)] = unique_list  # 将新键值对添加到新字典中
            tf[key] = [mid_key]
    # 更新原始字典
    tf.update(new_tf)
    # 检测是否到达转化完成的条件
    for key in tf:
        if len(tf[key]) != 1:
            mid_key_judgment = ''.join(tf[key])
            # print(mid_key_judgment)
            for word in alphabet:
                if (mid_key_judgment,word) in tf:
                    # print(mid_key_judgment)
                    tag = False
    if not tag:
        for key in tf:
            if len(tf[key]) != 1:
                mid_list = ''.join(set(''.join(tf[key])))
                tf[key] = [mid_list]
# 输出更新后的字典
print(transition_function_print(tf))
# print(tf)


#### 问题与解决：没有标识终止状态
- 太粗糙

In [84]:
print(accept_state)

In [16]:
import copy

tf = copy.deepcopy(transition_function)
mid_accept_state = copy.deepcopy(accept_state)
new_tf = {}  # 创建一个新的字典来保存修改后的键值对

tag = True
while tag:
    # tag = False
    for key in tf:
        if len(tf[key]) != 1:
            # tag = True
            mid_key = "".join(tf[key])  # 将值拼接成一个字符串作为新键
            mid_key_list = tf[key]  # 将值列表作为新键的值
            new_values = {}
            for item in alphabet:
                list1 = []
                for sub_key in mid_key_list:
                    try:
                        list1 += tf[(sub_key, item)]
                    except KeyError:
                        continue
                unique_list = list(set(list1))
                new_tf[(mid_key,item)] = unique_list  # 将新键值对添加到新字典中
            tf[key] = [mid_key]
    # 更新原始字典
    tf.update(new_tf)
    # 检测是否到达转化完成的条件
    for key in tf:
        if len(tf[key]) != 1:
            mid_key_judgment = ''.join(tf[key])
            # print(mid_key_judgment)
            for word in alphabet:
                if (mid_key_judgment,word) in tf:
                    # print(mid_key_judgment)
                    tag = False
    if not tag:
        for key in tf:
            if len(tf[key]) != 1:
                mid_list = ''.join(set(''.join(tf[key])))
                tf[key] = [mid_list]
        # 如果新键含有终止状态，将其放入终止状态表
        for key in tf:
            for item in tf[key]:
                # print(item)
                for item_s in item:
                    if item_s in accept_state:
                        mid_accept_state.append(item)
                
mid_accept_state = list(set(mid_accept_state))
mid_state = list(set(states+mid_accept_state))
# 输出更新后的字典
print(transition_function_print(tf))
print(tf)
print(alphabet)
print(mid_accept_state)
print(mid_state)
# print(tf)


In [121]:
fsm = construct_fsm(mid_state, alphabet, tf, start_state, mid_accept_state)


In [122]:
fsm = construct_fsm(mid_state, alphabet, tf, start_state, mid_accept_state)

# 测试字符串
test_strings = ['ba', 'babb', 'babba', 'babbbb', 'bb']

# 运行有穷状态自动机并输出结果
for s in test_strings:
    print(f"String '{s}' matches: {fsm.run(s)}")

## 总结当前未考虑的情况与限制
- 非终结符只有**单个**的
- 终结符只有**单个**的
- 未考虑**除样例外**的其余状况

#### 总结：未来尝试只使用索引代替中间态

### DFA简化

In [123]:
mid_tf = copy.deepcopy(tf)
mid_accept_state_2 = copy.deepcopy(mid_accept_state)
dynamic_accept_state = set() # 存储动态终止状态
dynamic_no_accept_state = set() # 存储动态非终止状态

# 初始化
for key in mid_tf:
    for item in mid_tf[key]:
        if item in mid_accept_state_2:
            dynamic_accept_state.add(item)
        else:
            dynamic_no_accept_state.add(item)
print(dynamic_accept_state)
print(dynamic_no_accept_state)

In [131]:
def minimize_dfa(states, alphabet, transition_function, final_states):
    # 初始划分，将终态和非终态分开
    partition = [set(final_states), set(states) - set(final_states)]
    # print(partition)

    while True:
        new_partition = []
        for group in partition:
            split_group = []
            for state in group:
                transitions = {}
                for symbol in alphabet:
                    try:
                        next_state = transition_function[(state, symbol)][0]
                    except KeyError:
                        continue
                    # 查找状态的转移目标所在的分组
                    for idx, part in enumerate(partition):
                        if next_state in part:
                            transitions[symbol] = idx
                            break
                split_group.append((state, transitions))

            # 根据转移目标所在的分组将状态分组
            groups = {}
            for state, trans in split_group:
                key = tuple(trans.values())
                if key not in groups:
                    groups[key] = [state]
                else:
                    groups[key].append(state)

            # 将分组结果加入到新分组列表中
            new_partition.extend(groups.values())

        # 如果新分组和旧组相同，则达到最小化
        if new_partition == partition:
            break
        else:
            partition = new_partition

    # 将等价状态替换为代表状态
    equivalent_states = {}
    for idx, group in enumerate(partition):
        for state in group:
            equivalent_states[state] = idx

    return equivalent_states


In [132]:
eqs = minimize_dfa(mid_state, alphabet, mid_tf, mid_accept_state)
print(eqs)

In [134]:
for key in eqs:
    for key_2 in eqs:
        if eqs[key] == eqs[key_2]:
            if key != key_2:
                for symbol in alphabet:
                    try:
                        del mid_tf[(key,symbol)]
                    except KeyError:
                        continue
print(mid_tf)

### 总结
暂时未考虑更多情况
- 五元组提取
- NFA到DFA
- DFA运行
- DFA最小化

### 未来展望：
- 使用更简便的数据结构
- 优化代码逻辑
- 将代码按功能打包
- 构造GUI交互窗口